In [ ]:
# ======================================================================
# 单元 3：使用最优参数训练并推理生成 submission.csv
# ======================================================================

import json
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from sklearn.preprocessing import StandardScaler



19
167.8s 118 model_x_val_list:[0.6287498120885685]

466.1s 342 model_x_val_list:[0.6876166711048914]

726.6s 538 model_x_val_list:[0.6191256934265665]

1066.2s 790 model_x_val_list:[0.6408800487589146]

1305.3s 971 model_x_val_list:[0.655929250511712]


42
1628.7s 1194 model_x_val_list:[0.6230685653714565]

1946.9s 1397 model_x_val_list:[0.6816828236837671]

2255.7s 1594 model_x_val_list:[0.6380264214753033]

2580.3s 1801 model_x_val_list:[0.634864329873299]

2924.6s 2020 model_x_val_list:[0.6709610554462989]


64
3383.7s 2263 model_x_val_list:[0.6499298537217154]

3844.3s 2455 model_x_val_list:[0.6796746843223926]

4334.1s 2660 model_x_val_list:[0.6356689155008671]

5013.2s 2941 model_x_val_list:[0.622604983644486]

5565.2s 3172 model_x_val_list:[0.6291349753625153]



364.2s 263 model_y_val_list: [0.561663060031395]

610.0s 450 model_y_val_list: [0.6099795133957178]

882.5s 653 model_y_val_list: [0.5821781166620897]

1194.2s 885 model_y_val_list: [0.5797950492099755]

1458.6s 1083 model_y_val_list: [0.6141496390569589]


1814.7s 1311 model_y_val_list: [0.5869238502917573]

2108.5s 1499 model_y_val_list: [0.6196303800034224]

2436.7s 1708 model_y_val_list: [0.5611591410229677]

2797.3s 1937 model_y_val_list: [0.58890986012815]

3141.5s 2156 model_y_val_list: [0.5733821356221309]


3647.3s 2371 model_y_val_list: [0.5816903462069517]

4140.2s 2578 model_y_val_list: [0.6027354114904219]

4627.9s 2782 model_y_val_list: [0.5441324146492953]

5284.7s 3054 model_y_val_list: [0.6025653593135782]

5806.7s 3271 model_y_val_list: [0.6014968684030821]

# Now all params are found with opt 

Seed 19: X values: [0.6682479069545363, 0.6271058951198512, 0.6420836529418633, 0.6315098601225932, 0.6088514363599971], Y values: [0.6105412800407741, 0.5685900346321365, 0.5800642574688976, 0.5424466438549035, 0.6065578838845658]
Seed 42: X values: [0.635550935708804, 0.6618808185229955, 0.6601690848073783, 0.655013706522254, 0.6353804208791102], Y values: [0.5996811457375933, 0.5767373171936827, 0.5709060747287746, 0.5777591002546802, 0.5833306324308463]
Seed 64: X values: [0.6522520906518646, 0.6523542004264561, 0.6150006706401064, 0.6201418607790589, 0.654220539020383], Y values: [0.5700290755981909, 0.5508604222290143, 0.5747010325064147, 0.5947516842287439, 0.620652449164692]
Seed 89: X values: [0.6522301035613204, 0.6347463148530911, 0.6482107260680275, 0.6229904400103146, 0.6171141605262076], Y values: [0.5757633951407591, 0.5740469305432565, 0.5831753725979005, 0.5951615811987084, 0.6084288929242135]


In [ ]:
# -------------------------------
# 定义模型与损失函数（与单元 2 相同）
# -------------------------------
class ResidualGRU(nn.Module):
    def __init__(self, input_dim, horizon, hidden_size=256, num_layers=2, dropout=0.1, nheads=4):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.ln = nn.LayerNorm(hidden_size)
        self.attn = nn.MultiheadAttention(hidden_size, nheads, batch_first=True)
        self.q = nn.Parameter(torch.randn(1,1,hidden_size))
        self.head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, horizon)
        )
    def forward(self, x):
        h, _ = self.gru(x)
        h = self.ln(h)
        B = h.size(0)
        q = self.q.expand(B,-1,-1)
        ctx, _ = self.attn(q, h, h)
        out = self.head(ctx.squeeze(1))
        return torch.cumsum(out, dim=1)

class TemporalHuberLoss(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.02, alpha=0.0):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
        self.alpha = alpha
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        huber = torch.where(abs_err <= self.delta,
                            0.5*err*err,
                            self.delta*(abs_err - 0.5*self.delta))
        L = pred.size(1)
        t = torch.arange(L, device=pred.device).float()
        decay = torch.exp(-self.time_decay * t).view(1,L)
        weights = (1.0 + t)**self.alpha * decay
        huber = huber * weights
        mask  = mask  * weights
        return (huber * mask).sum() / (mask.sum() + 1e-8)

# 构建批次函数（使用已填充数据）
def build_batches_padded(X, y, mask, batch_size):
    batches = []
    n = len(X)
    for i in range(0, n, batch_size):
        end = min(i+batch_size, n)
        xs = torch.tensor(np.stack(X[i:end]), dtype=torch.float32)
        ys = torch.tensor(np.stack(y[i:end]), dtype=torch.float32)
        ms = torch.tensor(np.stack(mask[i:end]), dtype=torch.float32)
        batches.append((xs, ys, ms))
    return batches
from tqdm.auto import tqdm
from tqdm.auto import tqdm

def train_residual_model_padded(
    X_tr, y_tr, mask_tr,
    X_va, y_va, mask_va,
    input_dim, horizon,
    hidden_size, num_layers,
    dropout, lr, batch_size,
    delta, time_decay, alpha,
    max_epochs=150, patience=20
):
    """
    带 tqdm + official RMSE 验证显示的版本
    输入输出接口与原函数完全一致
    """
    import torch
    import torch.nn as nn
    import numpy as np
    from tqdm import tqdm
    official_rmse_list = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ResidualGRU(input_dim, horizon, hidden_size, num_layers, dropout).to(device)
    criterion = TemporalHuberLoss(delta=delta, time_decay=time_decay, alpha=alpha)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

    tr_batches = build_batches_padded(X_tr, y_tr, mask_tr, batch_size)
    va_batches = build_batches_padded(X_va, y_va, mask_va, batch_size)

    best_loss, best_state, bad = float('inf'), None, 0

    for epoch in tqdm(range(1, max_epochs + 1), desc=f"Training GRU ({hidden_size})"):
        # ---------------------
        # 训练阶段
        # ---------------------
        model.train()
        train_losses = []
        for bx, by, bm in tr_batches:
            bx, by, bm = bx.to(device), by.to(device), bm.to(device)
            pred = model(bx)
            loss = criterion(pred, by, bm)
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())
        mean_tr = float(np.mean(train_losses))

        # ---------------------
        # 验证阶段
        # ---------------------
        model.eval()
        val_losses, preds_list, trues_list, masks_list = [], [], [], []
        with torch.no_grad():
            for bx, by, bm in va_batches:
                bx, by, bm = bx.to(device), by.to(device), bm.to(device)
                pred = model(bx)
                val_losses.append(criterion(pred, by, bm).item())
                preds_list.append(pred.cpu().numpy())
                trues_list.append(by.cpu().numpy())
                masks_list.append(bm.cpu().numpy())

        val_mean = float(np.mean(val_losses))
        scheduler.step(val_mean)

        # ---------------------
        # 额外：计算 Official RMSE
        # ---------------------
        try:
            preds_all = np.concatenate(preds_list, axis=0)
            trues_all = np.concatenate(trues_list, axis=0)
            masks_all = np.concatenate(masks_list, axis=0)

            mask_sum = np.sum(masks_all)
            if mask_sum > 0:
                mse = np.sum(((preds_all - trues_all) ** 2) * masks_all) / (mask_sum + 1e-8)
                official_rmse = float(np.sqrt(mse))
            else:
                official_rmse = np.nan
        except Exception as e:
            print(f"[Warning] Official RMSE计算出错: {e}")
            official_rmse = np.nan

        # ---------------------
        # Early Stopping
        # ---------------------
        if val_mean < best_loss:
            best_loss, bad = val_mean, 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            bad += 1
            if bad >= patience:
                print(f"Early stop at epoch {epoch}")
                break

        # ---------------------
        # 输出信息
        # ---------------------
        tqdm.write(f"Epoch {epoch:03d} | train={mean_tr:.4f} | val={val_mean:.4f} | offRMSE={official_rmse:.4f}")

    # ---------------------
    # 恢复最佳权重
    # ---------------------
    if best_state:
        official_rmse_list.append(official_rmse)
        model.load_state_dict(best_state)
    print(f"Official RMSE: {official_rmse_list} for this kfold model")
    return model,official_rmse_list   

In [ ]:
# ======================================================================
# 单元 2：多种子、多折模型训练与保存
# ======================================================================

import pickle
import json
import re
import torch
from pathlib import Path

# --- 1. Setup and Seed Discovery ---

# Define the cache directory where data and parameter files are stored
CACHE_DIR = Path("/kaggle/input/cache-feature-engineer-nfl")
SEED_DIR = Path("/kaggle/input/packages-json-params-nfl")
# Use regular expressions to extract seed numbers from filenames
seed_pattern = re.compile(r'seed_(\d+)')

# Find all parameter files and extract their seed numbers
param_files = {p.name: seed_pattern.search(p.name).group(1)
               for p in SEED_DIR.glob("best_params_seed_*.json") if seed_pattern.search(p.name)}

# Find all data files and extract their seed numbers
data_files = {p.name: seed_pattern.search(p.name).group(1)
              for p in CACHE_DIR.glob("folds_data_seed_*.pkl") if seed_pattern.search(p.name)}

# Identify the seeds that have BOTH a parameter file and a data file
valid_seeds = sorted(list(set(param_files.values()) & set(data_files.values())), key=int)

if not valid_seeds:
    raise FileNotFoundError("No matching seed files found for both params (*.json) and data (*.pkl). Please check the filenames in CACHE_DIR.")

print(f"✅ Found {len(valid_seeds)} valid seeds to process: {valid_seeds}\n")


# --- 2. Main Training Loop for Each Seed ---

# Dictionaries to store the trained models from all seeds
all_models_x = {}
all_models_y = {} # CORRECTED: Initialized all_models_y

# Define the main output directory for saved models
output_dir = Path("/kaggle/working/saved_models/")
output_dir.mkdir(exist_ok=True, parents=True)

# Iterate through each valid seed
for seed in valid_seeds:
    print(f"--- Starting training for SEED: {seed} ---")
    # Load the specific parameter file for the current seed
    param_filepath = SEED_DIR / f"best_params_seed_{seed}.json"
    with open(param_filepath, 'r') as f:
        best_params = json.load(f)
    print(f"Loaded parameters from: {param_filepath.name}")

    # Load the specific data file for the current seed
    data_filepath = CACHE_DIR / f"folds_data_seed_{seed}.pkl"
    with open(data_filepath, 'rb') as f:
        data = pickle.load(f)
    print(f"Loaded data from: {data_filepath.name}")

    # Unpack the data dictionary
    folds_data = data['folds_data']
    horizon    = data['horizon']
    feature_cols = data['feature_cols']
    idx_x = data['idx_x']
    idx_y = data['idx_y']

    # Lists to hold the models for the current seed's folds
    models_x_for_seed, models_y_for_seed = [], []

    # --- Inner loop to train on each fold ---
    for i, fd in enumerate(folds_data):
        print(f"\n--- Training Fold {i+1}/{len(folds_data)} for Seed {seed} ---")
        X_tr    = fd['X_train']
        X_va    = fd['X_val']
        ydx_tr  = fd['y_dx_train']
        ydx_va  = fd['y_dx_val']
        ydy_tr  = fd['y_dy_train']
        ydy_va  = fd['y_dy_val']
        m_tr    = fd['mask_train']
        m_va    = fd['mask_val']
        input_dim = X_tr[0].shape[-1]

        # Train Δx model
        model_x, model_x_val_list = train_residual_model_padded(
            X_tr, ydx_tr, m_tr, X_va, ydx_va, m_va,
            input_dim=input_dim, horizon=horizon, **best_params,
            max_epochs=180, patience=25
        )
        print(f"model_x_val_list:{model_x_val_list}")

        # Train Δy model
        model_y, model_y_val_list = train_residual_model_padded(
            X_tr, ydy_tr, m_tr, X_va, ydy_va, m_va,
            input_dim=input_dim, horizon=horizon, **best_params,
            max_epochs=180, patience=25
        )
        print(f"model_y_val_list: {model_y_val_list}")

        models_x_for_seed.append(model_x)
        models_y_for_seed.append(model_y)

    # Store the list of trained models for the current seed in the main dictionary
    all_models_x[seed] = models_x_for_seed
    all_models_y[seed] = models_y_for_seed
    print(f"\n✅ Finished training for Seed {seed}. Stored {len(models_x_for_seed)} fold models.")

    # --- 3. ADDED: Save the trained models for the current seed ---
    print(f"💾 Saving models for Seed {seed}...")
    seed_output_dir = output_dir / f"seed_{seed}"
    seed_output_dir.mkdir(exist_ok=True)

    for i, model in enumerate(models_x_for_seed):
        save_path = seed_output_dir / f"model_x_fold_{i}.pth"
        torch.save(model.state_dict(), save_path)

    for i, model in enumerate(models_y_for_seed):
        save_path = seed_output_dir / f"model_y_fold_{i}.pth"
        torch.save(model.state_dict(), save_path)

    print(f"Models saved to: {seed_output_dir}")


print("\n\n🎉 All training cycles complete for all seeds. 🎉")
print(f"Total seeds processed: {list(all_models_x.keys())}")
print(f"All model files saved in: {output_dir}")


In [ ]:
# -------------------------------
# 重新生成测试集数据并预测
# -------------------------------
# Feature engineering helpers are needed here
def unify_left_direction(df: pd.DataFrame) -> pd.DataFrame:
    if 'play_direction' not in df.columns:
        return df
    df = df.copy()
    right = df['play_direction'].eq('right')
    if 'x' in df.columns: df.loc[right, 'x'] = FIELD_LENGTH - df.loc[right, 'x']
    if 'y' in df.columns: df.loc[right, 'y'] = FIELD_WIDTH  - df.loc[right, 'y']
    for col in ('dir','o'):
        if col in df.columns:
            df.loc[right, col] = (df.loc[right, col] + 180.0) % 360.0
    if 'ball_land_x' in df.columns:
        df.loc[right, 'ball_land_x'] = FIELD_LENGTH - df.loc[right, 'ball_land_x']
    if 'ball_land_y' in df.columns:
        df.loc[right, 'ball_land_y'] = FIELD_WIDTH  - df.loc[right, 'ball_land_y']
    return df

# 读取 CSV
DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
test_input  = pd.read_csv(DATA_DIR / "test_input.csv")
test_template = pd.read_csv(DATA_DIR / "test.csv")

# %% [code]
# ======================================================================
# 单元 1：构建折数据并缓存
# ======================================================================

import os
import json
import hashlib
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

# -------------------------------
# 配置及常量
# -------------------------------
class Config:
    # 修改为 Kaggle 数据目录
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    CACHE_DIR = Path("/kaggle/working/")
    CACHE_DIR.mkdir(exist_ok=True, parents=True)

    WINDOW_SIZE = 10
    MAX_FUTURE_HORIZON = 94
    CV_BASELINE_L = 4
    N_FOLDS = 5
    FEATURE_GROUPS = [
        'distance_rate','target_alignment','multi_window_rolling','extended_lags',
        'velocity_changes','field_position','role_specific','time_features',
        'jerk_features',#'curvature_land_features'
    ]

# -------------------------------
# 基本辅助函数
# -------------------------------
YARDS_TO_METERS = 0.9144
FPS            = 10.0
FIELD_LENGTH   = 120.0
FIELD_WIDTH    = 53.3

def wrap_angle_deg(s):
    return ((s + 180.0) % 360.0) - 180.0

def unify_left_direction(df: pd.DataFrame) -> pd.DataFrame:
    if 'play_direction' not in df.columns:
        return df
    df = df.copy()
    right = df['play_direction'].eq('right')
    if 'x' in df.columns: df.loc[right, 'x'] = FIELD_LENGTH - df.loc[right, 'x']
    if 'y' in df.columns: df.loc[right, 'y'] = FIELD_WIDTH  - df.loc[right, 'y']
    for col in ('dir','o'):
        if col in df.columns:
            df.loc[right, col] = (df.loc[right, col] + 180.0) % 360.0
    if 'ball_land_x' in df.columns:
        df.loc[right, 'ball_land_x'] = FIELD_LENGTH - df.loc[right, 'ball_land_x']
    if 'ball_land_y' in df.columns:
        df.loc[right, 'ball_land_y'] = FIELD_WIDTH  - df.loc[right, 'ball_land_y']
    return df

# %% [code]
class FeatureEngineer:
    """
    Modular, ablation-friendly feature builder (pandas or cuDF pandas-API).
    """
    def __init__(self, feature_groups_to_create):
        self.gcols = ['game_id', 'play_id', 'nfl_id']
        self.active_groups = feature_groups_to_create
        self.feature_creators = {
            'distance_rate': self._create_distance_rate_features,
            'target_alignment': self._create_target_alignment_features,
            'multi_window_rolling': self._create_multi_window_rolling_features,
            'extended_lags': self._create_extended_lag_features,
            'velocity_changes': self._create_velocity_change_features,
            'field_position': self._create_field_position_features,
            'role_specific': self._create_role_specific_features,
            'time_features': self._create_time_features,
            'jerk_features': self._create_jerk_features,
            'curvature_land_features': self._create_curvature_land_features,
        }
        self.created_feature_cols = []

    def _height_to_feet(self, height_str):
        try:
            ft, inches = map(int, str(height_str).split('-'))
            return ft + inches / 12
        except Exception:
            return 6.0

    def _create_basic_features(self, df):
        print("Step 1/3: Adding basic features...")
        df = df.copy()
        df['player_height_feet'] = df['player_height'].apply(self._height_to_feet)

        # Correct kinematics: dir is from +x CCW
        dir_rad = np.deg2rad(df['dir'].fillna(0.0).astype('float32'))
        df['velocity_x']     = df['s'] * np.cos(dir_rad)
        df['velocity_y']     = df['s'] * np.sin(dir_rad)
        df['acceleration_x'] = df['a'] * np.cos(dir_rad)
        df['acceleration_y'] = df['a'] * np.sin(dir_rad)

        # Roles
        df['is_offense']  = (df['player_side'] == 'Offense').astype(np.int8)
        df['is_defense']  = (df['player_side'] == 'Defense').astype(np.int8)
        df['is_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(np.int8)
        df['is_coverage'] = (df['player_role'] == 'Defensive Coverage').astype(np.int8)
        df['is_passer']   = (df['player_role'] == 'Passer').astype(np.int8)

        # Energetics (consistent units)
        mass_kg = df['player_weight'].fillna(200.0) / 2.20462
        v_ms = df['s'] * YARDS_TO_METERS
        df['momentum_x'] = mass_kg * df['velocity_x'] * YARDS_TO_METERS
        df['momentum_y'] = mass_kg * df['velocity_y'] * YARDS_TO_METERS
        df['kinetic_energy'] = 0.5 * mass_kg * (v_ms ** 2)

        # Ball landing geometry (static)
        if {'ball_land_x','ball_land_y'}.issubset(df.columns):
            ball_dx = df['ball_land_x'] - df['x']
            ball_dy = df['ball_land_y'] - df['y']
            dist = np.hypot(ball_dx, ball_dy)
            df['distance_to_ball'] = dist
            inv = 1.0 / (dist + 1e-6)
            df['ball_direction_x'] = ball_dx * inv
            df['ball_direction_y'] = ball_dy * inv
            df['closing_speed'] = (
                df['velocity_x'] * df['ball_direction_x'] +
                df['velocity_y'] * df['ball_direction_y']
            )

        base = [
            'x','y','s','a','o','dir','frame_id','ball_land_x','ball_land_y',
            'player_height_feet','player_weight',
            'velocity_x','velocity_y','acceleration_x','acceleration_y',
            'momentum_x','momentum_y','kinetic_energy',
            'is_offense','is_defense','is_receiver','is_coverage','is_passer',
            'distance_to_ball','ball_direction_x','ball_direction_y','closing_speed'
        ]
        self.created_feature_cols.extend([c for c in base if c in df.columns])
        return df

    # ---- feature groups ----
    def _create_distance_rate_features(self, df):
        new_cols = []
        if 'distance_to_ball' in df.columns:
            d = df.groupby(self.gcols)['distance_to_ball'].diff()
            df['d2ball_dt']  = d.fillna(0.0) * FPS
            df['d2ball_ddt'] = df.groupby(self.gcols)['d2ball_dt'].diff().fillna(0.0) * FPS
            df['time_to_intercept'] = (df['distance_to_ball'] /
                                       (df['d2ball_dt'].abs() + 1e-3)).clip(0, 10)
            new_cols = ['d2ball_dt','d2ball_ddt','time_to_intercept']
        return df, new_cols

    def _create_target_alignment_features(self, df):
        new_cols = []
        if {'ball_direction_x','ball_direction_y','velocity_x','velocity_y'}.issubset(df.columns):
            df['velocity_alignment'] = df['velocity_x']*df['ball_direction_x'] + df['velocity_y']*df['ball_direction_y']
            df['velocity_perpendicular'] = df['velocity_x']*(-df['ball_direction_y']) + df['velocity_y']*df['ball_direction_x']
            new_cols.extend(['velocity_alignment','velocity_perpendicular'])
            if {'acceleration_x','acceleration_y'}.issubset(df.columns):
                df['accel_alignment'] = df['acceleration_x']*df['ball_direction_x'] + df['acceleration_y']*df['ball_direction_y']
                new_cols.append('accel_alignment')
        return df, new_cols

    def _create_multi_window_rolling_features(self, df):
        # keep it simple & compatible (works with cuDF pandas-API); vectorized rolling per group
        new_cols = []
        for window in (3, 5, 10):
            for col in ('velocity_x','velocity_y','s','a'):
                if col in df.columns:
                    r_mean = df.groupby(self.gcols)[col].rolling(window, min_periods=1).mean()
                    r_std  = df.groupby(self.gcols)[col].rolling(window, min_periods=1).std()
                    # align indices
                    r_mean = r_mean.reset_index(level=list(range(len(self.gcols))), drop=True)
                    r_std  = r_std.reset_index(level=list(range(len(self.gcols))), drop=True)
                    df[f'{col}_roll{window}'] = r_mean
                    df[f'{col}_std{window}']  = r_std.fillna(0.0)
                    new_cols.extend([f'{col}_roll{window}', f'{col}_std{window}'])
        return df, new_cols

    def _create_extended_lag_features(self, df):
        new_cols = []
        for lag in (1,2,3,4,5):
            for col in ('x','y','velocity_x','velocity_y'):
                if col in df.columns:
                    g = df.groupby(self.gcols)[col]
                    lagv = g.shift(lag)
                    # safe fill for first frames (no "future" leakage)
                    df[f'{col}_lag{lag}'] = lagv.fillna(g.transform('first'))
                    new_cols.append(f'{col}_lag{lag}')
        return df, new_cols

    def _create_velocity_change_features(self, df):
        new_cols = []
        if 'velocity_x' in df.columns:
            df['velocity_x_change'] = df.groupby(self.gcols)['velocity_x'].diff().fillna(0.0)
            df['velocity_y_change'] = df.groupby(self.gcols)['velocity_y'].diff().fillna(0.0)
            df['speed_change']      = df.groupby(self.gcols)['s'].diff().fillna(0.0)
            d = df.groupby(self.gcols)['dir'].diff().fillna(0.0)
            df['direction_change']  = wrap_angle_deg(d)
            new_cols = ['velocity_x_change','velocity_y_change','speed_change','direction_change']
        return df, new_cols

    def _create_field_position_features(self, df):
        df['dist_from_left'] = df['y']
        df['dist_from_right'] = FIELD_WIDTH - df['y']
        df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
        df['dist_from_endzone']  = np.minimum(df['x'], FIELD_LENGTH - df['x'])
        return df, ['dist_from_sideline','dist_from_endzone']

    def _create_role_specific_features(self, df):
        new_cols = []
        if {'is_receiver','velocity_alignment'}.issubset(df.columns):
            df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
            df['receiver_deviation']  = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0.0))
            new_cols.extend(['receiver_optimality','receiver_deviation'])
        if {'is_coverage','closing_speed'}.issubset(df.columns):
            df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']
            new_cols.append('defender_closing_speed')
        return df, new_cols

    def _create_time_features(self, df):
        df['frames_elapsed']  = df.groupby(self.gcols).cumcount()
        df['normalized_time'] = df.groupby(self.gcols)['frames_elapsed'].transform(
            lambda x: x / (x.max() + 1e-9)
        )
        return df, ['frames_elapsed','normalized_time']

    def _create_jerk_features(self, df):
        new_cols = []
        if 'a' in df.columns:
            df['jerk'] = df.groupby(self.gcols)['a'].diff().fillna(0.0) * FPS
            new_cols.append('jerk')
        if {'acceleration_x','acceleration_y'}.issubset(df.columns):
            df['jerk_x'] = df.groupby(self.gcols)['acceleration_x'].diff().fillna(0.0) * FPS
            df['jerk_y'] = df.groupby(self.gcols)['acceleration_y'].diff().fillna(0.0) * FPS
            new_cols.extend(['jerk_x','jerk_y'])
        return df, new_cols
    def _create_curvature_land_features(self, df):
        """
        -落点侧向偏差（符号）：landing_point 相对“当前运动方向”的左右偏离
          lateral = cross(u_dir, vector_to_land)（>0 表示落点在运动方向左侧）
        -bearing_to_land_signed: 运动方向 vs 落点方位角
        -速度归一化曲率： wrap(Δdir)/ (s*Δt) ，窗口化(3/5) 的均值/绝对值
        """
        import numpy as np
        # 侧向偏差 & bearing_to_land
        if {'ball_land_x','ball_land_y'}.issubset(df.columns):
            dx = df['ball_land_x'] - df['x']
            dy = df['ball_land_y'] - df['y']
            bearing = np.arctan2(dy, dx)
            a_dir = np.deg2rad(df['dir'].fillna(0.0).values)
            # 有符号方位差
            df['bearing_to_land_signed'] = np.rad2deg(np.arctan2(np.sin(bearing - a_dir), np.cos(bearing - a_dir)))
            # 侧向偏差：d × u (2D cross, z 分量)
            ux, uy = np.cos(a_dir), np.sin(a_dir)
            df['land_lateral_offset'] = dy*ux - dx*uy  # >0 落点在左侧
    
        # 曲率（按序列）
        ddir = df.groupby(self.gcols)['dir'].diff().fillna(0.0)
        ddir = ((ddir + 180.0) % 360.0) - 180.0
        curvature = np.deg2rad(ddir).astype('float32') / (df['s'].replace(0, np.nan).astype('float32') * 0.1 + 1e-6)
        df['curvature_signed'] = curvature.fillna(0.0)
        df['curvature_abs'] = df['curvature_signed'].abs()
    
        # 窗口均值（3/5）
        for w in (3,5):
            r = df.groupby(self.gcols)['curvature_signed'].rolling(w, min_periods=1).mean().reset_index(level=[0,1,2], drop=True)
            df[f'curv_signed_roll{w}'] = r
            r2 = df.groupby(self.gcols)['curvature_abs'].rolling(w, min_periods=1).mean().reset_index(level=[0,1,2], drop=True)
            df[f'curv_abs_roll{w}'] = r2
    
        new_cols = ['bearing_to_land_signed','land_lateral_offset',
                    'curvature_signed','curvature_abs','curv_signed_roll3','curv_abs_roll3',
                    'curv_signed_roll5','curv_abs_roll5']
        return df, [c for c in new_cols if c in df.columns]

    def transform(self, df):
        df = df.copy().sort_values(['game_id','play_id','nfl_id','frame_id'])
        df = self._create_basic_features(df)

        print("\nStep 2/3: Adding selected advanced features...")
        for group_name in self.active_groups:
            if group_name in self.feature_creators:
                creator = self.feature_creators[group_name]
                df, new_cols = creator(df)
                self.created_feature_cols.extend(new_cols)
                print(f"  [+] Added '{group_name}' ({len(new_cols)} cols)")
            else:
                print(f"  [!] Unknown feature group: {group_name}")

        final_cols = sorted(set(self.created_feature_cols))
        print(f"\nTotal features created: {len(final_cols)}")
        return df, final_cols

# %% [code]
# -------------------------------
# 序列生成函数
# -------------------------------
def build_play_direction_map(df_in: pd.DataFrame) -> pd.Series:
    s = (df_in[['game_id','play_id','play_direction']]
         .drop_duplicates()
         .set_index(['game_id','play_id'])['play_direction'])
    return s

def apply_direction_to_df(df: pd.DataFrame, dir_map: pd.Series) -> pd.DataFrame:
    if 'play_direction' not in df.columns:
        dir_df = dir_map.reset_index()
        df = df.merge(dir_df, on=['game_id','play_id'], how='left', validate='many_to_one')
    return unify_left_direction(df)

def prepare_sequences_with_advanced_features(
    input_df, output_df=None, test_template=None, is_training=True,
    window_size=10, feature_groups=None
):
    if feature_groups is None:
        feature_groups = Config.FEATURE_GROUPS
    dir_map = build_play_direction_map(input_df)
    input_df_u = unify_left_direction(input_df)
    if is_training:
        out_u = apply_direction_to_df(output_df, dir_map)
        target_rows  = out_u
        target_groups = out_u[['game_id','play_id','nfl_id']].drop_duplicates()
    else:
        if 'play_direction' not in test_template.columns:
            dir_df = dir_map.reset_index()
            test_template = test_template.merge(dir_df, on=['game_id','play_id'], how='left', validate='many_to_one')
        target_rows  = test_template
        target_groups = test_template[['game_id','play_id','nfl_id','play_direction']].drop_duplicates()

    assert target_rows[['game_id','play_id','play_direction']].isna().sum().sum() == 0

    fe = FeatureEngineer(feature_groups)
    processed_df, feature_cols = fe.transform(input_df_u)
    processed_df = processed_df.set_index(['game_id','play_id','nfl_id']).sort_index()
    grouped = processed_df.groupby(level=['game_id','play_id','nfl_id'])
    idx_x = feature_cols.index('x')
    idx_y = feature_cols.index('y')

    sequences, targets_dx, targets_dy, seq_meta = [], [], [], []
    for row in tqdm(list(target_groups.itertuples(index=False)), total=len(target_groups), desc="Creating sequences"):
        gid = row[0]; pid = row[1]; nid = row[2]
        play_dir = row[3] if (not is_training and len(row) >= 4) else None
        key = (gid,pid,nid)
        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue
        input_window = group_df.tail(window_size)
        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        if np.isnan(seq).any():
            if is_training:
                continue
            seq = np.nan_to_num(seq, nan=0.0)
        sequences.append(seq)
        if is_training:
            out_grp = target_rows[(target_rows['game_id']==gid) & (target_rows['play_id']==pid) & (target_rows['nfl_id']==nid)].sort_values('frame_id')
            if len(out_grp)==0:
                continue
            last_x = seq[-1, idx_x]
            last_y = seq[-1, idx_y]
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            targets_dx.append(dx.astype(np.float32))
            targets_dy.append(dy.astype(np.float32))
        seq_meta.append({
            'game_id': gid,
            'play_id': pid,
            'nfl_id': nid,
            'frame_id': int(input_window.iloc[-1]['frame_id']),
            'play_direction': (None if is_training else play_dir)
        })
    if is_training:
        return sequences, targets_dx, targets_dy, seq_meta, feature_cols, idx_x, idx_y
    return sequences, seq_meta, feature_cols, idx_x, idx_y
    sequences, targets_dx, targets_dy, seq_meta = [], [], [], []
    for row in tqdm(list(target_groups.itertuples(index=False)), total=len(target_groups), desc="Creating sequences"):
        gid = row[0]; pid = row[1]; nid = row[2]
        play_dir = row[3] if (not is_training and len(row) >= 4) else None
        key = (gid,pid,nid)
        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue
        input_window = group_df.tail(window_size)
        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        if np.isnan(seq).any():
            if is_training:
                continue
            seq = np.nan_to_num(seq, nan=0.0)
        sequences.append(seq)
        if is_training:
            out_grp = target_rows[(target_rows['game_id']==gid) & (target_rows['play_id']==pid) & (target_rows['nfl_id']==nid)].sort_values('frame_id')
            if len(out_grp)==0:
                continue
            last_x = seq[-1, idx_x]
            last_y = seq[-1, idx_y]
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            targets_dx.append(dx.astype(np.float32))
            targets_dy.append(dy.astype(np.float32))
        seq_meta.append({
            'game_id': gid,
            'play_id': pid,
            'nfl_id': nid,
            'frame_id': int(input_window.iloc[-1]['frame_id']),
            'play_direction': (None if is_training else play_dir)
        })
    if is_training:
        return sequences, targets_dx, targets_dy, seq_meta, feature_cols, idx_x, idx_y
    return sequences, seq_meta, feature_cols, idx_x, idx_y
    sequences, targets_dx, targets_dy, seq_meta = [], [], [], []
    for row in tqdm(list(target_groups.itertuples(index=False)), total=len(target_groups), desc="Creating sequences"):
        gid = row[0]; pid = row[1]; nid = row[2]
        play_dir = row[3] if (not is_training and len(row) >= 4) else None
        key = (gid,pid,nid)
        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue
        input_window = group_df.tail(window_size)
        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        if np.isnan(seq).any():
            if is_training:
                continue
            seq = np.nan_to_num(seq, nan=0.0)
        sequences.append(seq)
        if is_training:
            out_grp = target_rows[(target_rows['game_id']==gid) & (target_rows['play_id']==pid) & (target_rows['nfl_id']==nid)].sort_values('frame_id')
            if len(out_grp)==0:
                continue
            last_x = seq[-1, idx_x]
            last_y = seq[-1, idx_y]
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            targets_dx.append(dx.astype(np.float32))
            targets_dy.append(dy.astype(np.float32))
        seq_meta.append({
            'game_id': gid,
            'play_id': pid,
            'nfl_id': nid,
            'frame_id': int(input_window.iloc[-1]['frame_id']),
            'play_direction': (None if is_training else play_dir)
        })
    if is_training:
        return sequences, targets_dx, targets_dy, seq_meta, feature_cols, idx_x, idx_y
    return sequences, seq_meta, feature_cols, idx_x, idx_y

    # -------------------------------
# 常速基线函数
# -------------------------------
def cv_baseline_from_seq(seq_xy: np.ndarray, idx_x: int, idx_y: int, horizon: int, L: int = 4):
    x = seq_xy[:, idx_x].astype(np.float32)
    y = seq_xy[:, idx_y].astype(np.float32)
    T = len(x)
    L = max(2, min(L, T))
    if T >= 2:
        vx = np.diff(x)[-L+1:].mean()
        vy = np.diff(y)[-L+1:].mean()
    else:
        vx = 0.0
        vy = 0.0
    steps = np.arange(1, horizon+1, dtype=np.float32)
    dx_cv = vx * steps
    dy_cv = vy * steps
    return dx_cv, dy_cv
# pasted directly from the first notebook

In [ ]:
# ======================================================================
# 单元 4：多种子、多折模型推理与提交
# ======================================================================

print("🚀 Starting inference process...")

# --- 1. Generate Test Sequences (run once) ---
# This part is the same as before.
dir_map = build_play_direction_map(test_input)
test_input_u = unify_left_direction(test_input)

test_seqs, test_meta, feature_cols, idx_x, idx_y = prepare_sequences_with_advanced_features(
    test_input,
    output_df=None,
    test_template=test_template,
    is_training=False,
    window_size=Config.WINDOW_SIZE,
    feature_groups=Config.FEATURE_GROUPS
)

# --- 2. Calculate Constant Velocity Baseline (run once) ---
dx_cv_test = []
dy_cv_test = []
# Note: The 'horizon' variable is loaded within the loop, so we need to get it from the last loaded data.
# It should be the same for all seeds.
for seq in test_seqs:
    dx_cv, dy_cv = cv_baseline_from_seq(seq, idx_x, idx_y, horizon, L=Config.CV_BASELINE_L)
    dx_cv_test.append(dx_cv)
    dy_cv_test.append(dy_cv)

# --- 3. Scale Sequences and Predict with All Models ---
all_dx_res, all_dy_res = [], []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Outer loop: Iterate through each seed
for seed in valid_seeds:
    print(f"\\n--- Inferring with models from Seed {seed} ---")

    # Load the data file for the current seed to get scaler info
    data_filepath = CACHE_DIR / f"folds_data_seed_{seed}.pkl"
    with open(data_filepath, 'rb') as f:
        data = pickle.load(f)
    folds_data = data['folds_data']  # This has the scalers for this seed

    # Retrieve the models trained for this specific seed
    models_for_this_seed_x = all_models_x[seed]
    models_for_this_seed_y = all_models_y[seed]

    # Inner loop: Iterate through each fold within the seed
    for fold_idx, fd in enumerate(folds_data):
        print(f"  -> Using Fold {fold_idx+1}/{len(folds_data)} scaler and model")

        # Reconstruct the scaler for this specific fold
        scaler = StandardScaler()
        scaler.mean_ = fd['scaler_mean']
        scaler.scale_ = fd['scaler_scale']
        scaler.var_ = scaler.scale_**2
        scaler.n_features_in_ = len(scaler.mean_)

        # Scale the test sequences with this fold's scaler
        X_test_scaled = [scaler.transform(s).astype(np.float32) for s in test_seqs]

        # Get the corresponding model for this fold
        model_x = models_for_this_seed_x[fold_idx].eval().to(device)
        model_y = models_for_this_seed_y[fold_idx].eval().to(device)

        # Perform inference
        with torch.no_grad():
            X_t = torch.tensor(np.stack(X_test_scaled), dtype=torch.float32).to(device)
            dx_res = model_x(X_t).cpu().numpy()
            dy_res = model_y(X_t).cpu().numpy()

        # Append the results from this single model to the master list
        all_dx_res.append(dx_res)
        all_dy_res.append(dy_res)

# --- 4. Average Predictions and Generate Submission ---
print(f"\\n✅ Inference complete. Averaging results from {len(all_dx_res)} models...")

# Average residuals from ALL models (all folds across all seeds)
avg_dx_res = np.mean(all_dx_res, axis=0)
avg_dy_res = np.mean(all_dy_res, axis=0)

# Add the baseline back to the averaged residuals
dx_full = avg_dx_res + np.stack(dx_cv_test)
dy_full = avg_dy_res + np.stack(dy_cv_test)

# Generate submission file (this part is the same as before)
rows = []
FIELD_LENGTH = 120.0
FIELD_WIDTH  = 53.3
tt_idx = test_template.set_index(['game_id','play_id','nfl_id']).sort_index()

for i, meta in enumerate(test_meta):
    gid, pid, nid = meta['game_id'], meta['play_id'], meta['nfl_id']
    play_dir = meta.get('play_direction')
    play_is_right = (play_dir == 'right')

    try:
        fids = tt_idx.loc[(gid,pid,nid),'frame_id']
        if isinstance(fids, pd.Series):
            fids = fids.sort_values().tolist()
        else:
            fids = [int(fids)]
    except KeyError:
        continue

    last_x = test_seqs[i][-1, idx_x]
    last_y = test_seqs[i][-1, idx_y]

    for t, fid in enumerate(fids):
        tt = min(t, horizon - 1)
        x_uni = np.clip(last_x + dx_full[i, tt], 0, FIELD_LENGTH)
        y_uni = np.clip(last_y + dy_full[i, tt], 0, FIELD_WIDTH)

        if play_is_right:
            x_out = FIELD_LENGTH - x_uni
            y_out = FIELD_WIDTH  - y_uni
        else:
            x_out = x_uni
            y_out = y_uni

        rows.append({
            'id': f"{gid}_{pid}_{nid}_{int(fid)}",
            'x': float(x_out),
            'y': float(y_out)
        })

submission = pd.DataFrame(rows)
submission.to_csv("submission.csv", index=False)
print(f"\\n🎉 Saved submission.csv with {len(submission)} rows. 🎉")

